In [17]:
import socket
import struct
import time

IP = "127.0.0.1"
PORT_SERVER = 27015
PORT_CLIENT = 27016

sock_send = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock_recv = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock_recv.bind((IP, PORT_CLIENT))

# Set up which memory locations will be written to regularly,
# and which data type it expects (OFFSET:data type;)
control = "C;310A:c;089A:s;"
packet = bytes(control, "ascii")
sock_send.sendto(packet, (IP, PORT_SERVER))
for i in range(11):
	# 4th bit controls throttle
	# 00001000 = 8
	inputs = 8
	# Throttle goes from -4096 to 16384
	throttle = 16384/10*i
	# Pack both values into bytes, always send float64
	packet = struct.pack("<dd", inputs, throttle)
	sock_send.sendto(packet, (IP, PORT_SERVER))
	time.sleep(0.1);

# Set up which memory locations to monitor and receive regular updates from
monitor = "M;2B8:i;"
packet = bytes(monitor, "ascii")
sock_send.sendto(packet, (IP, PORT_SERVER))
for i in range(11):
	data, addr = sock_recv.recvfrom(1024)
	# Unpack bytes received, as float64
	values = struct.unpack("<d", data)
	print("TAS: " + str(values[0]/128) + " knots")


TAS: 12.21875 knots
TAS: 12.4921875 knots
TAS: 12.6171875 knots
TAS: 12.890625 knots
TAS: 13.1875 knots
TAS: 13.4453125 knots
TAS: 13.6015625 knots
TAS: 13.7421875 knots
TAS: 13.8828125 knots
TAS: 14.078125 knots
TAS: 14.2890625 knots


In [72]:
import socket
import struct
import time

IP = "127.0.0.1"
PORT_SERVER = 27015
PORT_CLIENT = 27016

sock_send = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock_recv = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
sock_recv.bind((IP, PORT_CLIENT))

# Set up which memory locations will be written to regularly,
# and which data type it expects (OFFSET:data type;)
control = "W;262:us;"
packet2 = bytes(control, "ascii")
data = struct.pack("<d", 0)
print(packet2 + data)
sock_send.sendto(packet2 + data, (IP, PORT_SERVER))


b'W;262:us;\x00\x00\x00\x00\x00\x00\xf0?'


17